### Deliverable 3b: Optimize the Model, keeping optimizations from 3a

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")

In [2]:
# Drop the non-beneficial features
application_df = application_df.drop(columns=["EIN","NAME","STATUS","SPECIAL_CONSIDERATIONS"])

In [3]:
# bin the APPLICATION_TYPE feature
type_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = list(type_counts[type_counts<200].index)
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# bin the CLASSIFICATION feature
class_counts = application_df.CLASSIFICATION.value_counts()
replace_class = list(class_counts[class_counts<1000].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes=='object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE    9
AFFILIATION         6
CLASSIFICATION      6
USE_CASE            5
ORGANIZATION        4
INCOME_AMT          9
dtype: int64

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list 
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, axis=1)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 5
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1,
    input_dim=number_input_features, 
    activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 205       
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints2/",exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:03d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

### Optimization: double the number of epochs for training

In [11]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 1s 850us/step - loss: 0.6517 - accuracy: 0.6019
Epoch 2/200
804/804 [==============================] - 1s 897us/step - loss: 0.5816 - accuracy: 0.7217
Epoch 3/200
804/804 [==============================] - 1s 892us/step - loss: 0.5657 - accuracy: 0.7234
Epoch 4/200
804/804 [==============================] - 1s 890us/step - loss: 0.5609 - accuracy: 0.7235
Epoch 5/200
742/804 [==========================>...] - ETA: 0s - loss: 0.5613 - accuracy: 0.7224
Epoch 5: saving model to checkpoints2\weights.005.hdf5
804/804 [==============================] - 1s 826us/step - loss: 0.5584 - accuracy: 0.7245
Epoch 6/200
804/804 [==============================] - 1s 885us/step - loss: 0.5573 - accuracy: 0.7243
Epoch 7/200
804/804 [==============================] - 1s 890us/step - loss: 0.5565 - accuracy: 0.7258
Epoch 8/200
804/804 [==============================] - 1s 844us/step - loss: 0.5554 - accuracy: 0.7276
Epoch 9/200
804/804 [===============

804/804 [==============================] - 1s 819us/step - loss: 0.5484 - accuracy: 0.7339
Epoch 64/200
804/804 [==============================] - 1s 835us/step - loss: 0.5484 - accuracy: 0.7340
Epoch 65/200
763/804 [===========================>..] - ETA: 0s - loss: 0.5484 - accuracy: 0.7347
Epoch 65: saving model to checkpoints2\weights.065.hdf5
804/804 [==============================] - 1s 865us/step - loss: 0.5485 - accuracy: 0.7345
Epoch 66/200
804/804 [==============================] - 1s 849us/step - loss: 0.5483 - accuracy: 0.7344
Epoch 67/200
804/804 [==============================] - 1s 862us/step - loss: 0.5483 - accuracy: 0.7345
Epoch 68/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7343
Epoch 69/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7341
Epoch 70/200
761/804 [===========================>..] - ETA: 0s - loss: 0.5490 - accuracy: 0.7336
Epoch 70: saving model to checkpoints2\weights.070

764/804 [===========================>..] - ETA: 0s - loss: 0.5475 - accuracy: 0.7347
Epoch 125: saving model to checkpoints2\weights.125.hdf5
804/804 [==============================] - 1s 809us/step - loss: 0.5474 - accuracy: 0.7349
Epoch 126/200
804/804 [==============================] - 1s 836us/step - loss: 0.5473 - accuracy: 0.7352
Epoch 127/200
804/804 [==============================] - 1s 786us/step - loss: 0.5474 - accuracy: 0.7346
Epoch 128/200
804/804 [==============================] - 1s 727us/step - loss: 0.5476 - accuracy: 0.7347
Epoch 129/200
804/804 [==============================] - 1s 774us/step - loss: 0.5477 - accuracy: 0.7346
Epoch 130/200
765/804 [===========================>..] - ETA: 0s - loss: 0.5492 - accuracy: 0.7329
Epoch 130: saving model to checkpoints2\weights.130.hdf5
804/804 [==============================] - 1s 800us/step - loss: 0.5476 - accuracy: 0.7345
Epoch 131/200
804/804 [==============================] - 1s 865us/step - loss: 0.5473 - accuracy: 0.

Epoch 186/200
804/804 [==============================] - 1s 801us/step - loss: 0.5470 - accuracy: 0.7354
Epoch 187/200
804/804 [==============================] - 1s 805us/step - loss: 0.5472 - accuracy: 0.7343
Epoch 188/200
804/804 [==============================] - 1s 894us/step - loss: 0.5470 - accuracy: 0.7344
Epoch 189/200
804/804 [==============================] - 1s 871us/step - loss: 0.5472 - accuracy: 0.7343
Epoch 190/200
802/804 [============================>.] - ETA: 0s - loss: 0.5468 - accuracy: 0.7346
Epoch 190: saving model to checkpoints2\weights.190.hdf5
804/804 [==============================] - 1s 889us/step - loss: 0.5470 - accuracy: 0.7345
Epoch 191/200
804/804 [==============================] - 1s 873us/step - loss: 0.5468 - accuracy: 0.7341
Epoch 192/200
804/804 [==============================] - 1s 845us/step - loss: 0.5469 - accuracy: 0.7349
Epoch 193/200
804/804 [==============================] - 1s 897us/step - loss: 0.5469 - accuracy: 0.7349
Epoch 194/200
804/

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7268 - 268ms/epoch - 1ms/step
Loss: 0.5529004335403442, Accuracy: 0.7267638444900513


In [13]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity-Optimization2.h5")